In [7]:
os
import logging
from dotenv import load_dotenv
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from IPython.display import Image, display

# Load environment variables
load_dotenv()



# Ensure API keys are loaded
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

  

os.environ["GROQ_API_KEY"] = GROQ_API_KEY

# Initialize LLM
llm = ChatGroq(model="qwen-2.5-32b")


# Define data models
class Analyst(BaseModel):
    affiliation: str = Field(description="Primary affiliation of the analyst.")
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="Role of the analyst in the context of the topic.")
    description: str = Field(description="Description of the analyst's focus, concerns, and motives.")

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}\n"


class Perspectives(BaseModel):
    analysts: List[Analyst] = Field(description="Comprehensive list of analysts with their roles and affiliations.")


class GenerateAnalystsState(TypedDict):
    topic: str  # Research topic
    max_analysts: int  # Number of analysts
    human_analyst_feedback: str  # Human feedback
    analysts: List[Analyst]  # List of analysts


# Instruction template
analyst_instructions = """You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:

1. First, review the research topic:
{topic}
        
2. Examine any editorial feedback that has been optionally provided to guide creation of the analysts: 
{human_analyst_feedback}
    
3. Determine the most interesting themes based upon documents and/or feedback above.
                    
4. Pick the top {max_analysts} themes.

5. Assign one analyst to each theme.
"""


def create_analysts(state: GenerateAnalystsState):
    """ Create analysts based on the given topic and feedback """

    topic = state['topic']
    max_analysts = state['max_analysts']
    human_analyst_feedback = state.get('human_analyst_feedback', '')

    # Enforce structured output
    structured_llm = llm.with_structured_output(Perspectives)

    # System message
    system_message = analyst_instructions.format(topic=topic,
                                                 human_analyst_feedback=human_analyst_feedback,
                                                 max_analysts=max_analysts)

    analysts = structured_llm.invoke(
            [SystemMessage(content=system_message)] + [HumanMessage(content="Generate the set of analysts.")]
        )
    

    return {"analysts": analysts.analysts}


def human_feedback(state: GenerateAnalystsState):
    """ No-op node that should be interrupted on """
    pass


def should_continue(state: GenerateAnalystsState):
    """ Return the next node to execute """

    # Check if human feedback exists
    human_analyst_feedback = state.get('human_analyst_feedback', None)
    if human_analyst_feedback:
        return "create_analysts"

    return END


# Build LangGraph
builder = StateGraph(GenerateAnalystsState)
builder.add_node("create_analysts", create_analysts)
builder.add_node("human_feedback", human_feedback)
builder.add_edge(START, "create_analysts")
builder.add_edge("create_analysts", "human_feedback")
builder.add_conditional_edges("human_feedback", should_continue, ["create_analysts", END])

# Compile the graph
memory = MemorySaver()
graph = builder.compile(interrupt_before=['human_feedback'], checkpointer=memory)



# Input data
max_analysts = 3
topic = "The benefits of adopting LangGraph as an agent framework"
thread = {"configurable": {"thread_id": "1"}}


for event in graph.stream({"topic": topic, "max_analysts": max_analysts}, thread, stream_mode="values"):
    analysts = event.get("analysts", [])
    if analysts:
        for analyst in analysts:
            print(f"\nName: {analyst.name}\nAffiliation: {analyst.affiliation}\nRole: {analyst.role}\nDescription: {analyst.description}")
            print("-" * 50)

# Get current state
state = graph.get_state(thread)

# Update state with human feedback
human_feedback_input = "Add in someone from a startup to add an entrepreneur perspective"
graph.update_state(thread, {"human_analyst_feedback": human_feedback_input}, as_node="human_feedback")


for event in graph.stream(None, thread, stream_mode="values"):
    analysts = event.get("analysts", [])
    if analysts:
        for analyst in analysts:
            print(f"\nName: {analyst.name}\nAffiliation: {analyst.affiliation}\nRole: {analyst.role}\nDescription: {analyst.description}")
            print("-" * 50)

# Final execution to end
further_feedback = None
if further_feedback:
    graph.update_state(thread, {"human_analyst_feedback": further_feedback}, as_node="human_feedback")

print("Final execution...")
for event in graph.stream(None, thread, stream_mode="updates"):
    print(f"--Node--\n{next(iter(event.keys()))}")

# Retrieve final state
final_state = graph.get_state(thread)
analysts = final_state.values.get('analysts', [])

# Display final analyst personas
print("Final Analyst Personas:")
if analysts:
    for analyst in analysts:
        print(f"\nName: {analyst.name}\nAffiliation: {analyst.affiliation}\nRole: {analyst.role}\nDescription: {analyst.description}")
        print("-" * 50)

print("Execution complete.")


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



Name: Dr. Emily Carter
Affiliation: NeuralNet Inc.
Role: Tech Strategist
Description: Dr. Emily Carter is a tech strategist deeply interested in the strategic implications of adopting new agent frameworks. She is particularly focused on how LangGraph can transform traditional agent systems into more sophisticated, interconnected frameworks.
--------------------------------------------------

Name: John Lee
Affiliation: Digital Rights United
Role: Data Privacy Advocate
Description: John Lee is a data privacy advocate who is concerned about the privacy implications of new AI technologies. He focuses on how LangGraph, as an agent framework, can be designed to respect user privacy and comply with data protection laws.
--------------------------------------------------

Name: Samantha Chen
Affiliation: Ethics in Tech Institute
Role: AI Ethics Researcher
Description: Samantha Chen is an AI ethics researcher who examines the ethical implications of AI technologies. She is interested in explo

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



Name: Dr. Emily Carter
Affiliation: Tech Innovation Institute
Role: Tech Evangelist
Description: Dr. Emily Carter is passionate about the latest advancements in technology and how they can revolutionize traditional industries. She focuses on the transformative potential of new tech solutions like LangGraph in enhancing agent frameworks.
--------------------------------------------------

Name: Samuel Lee
Affiliation: NextGen Startups Inc.
Role: Chief Technology Officer
Description: Samuel Lee is a startup veteran with a focus on rapid prototyping and agile methodologies. He is interested in how LangGraph can be leveraged to build scalable and efficient agent systems that can give startups a competitive edge.
--------------------------------------------------

Name: Professor Jonathan Reed
Affiliation: University of Tech Ethics
Role: AI Ethics Scholar
Description: Professor Jonathan Reed is deeply concerned with the ethical implications of AI. He examines how the adoption of LangGraph 

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


--Node--
create_analysts
--Node--
__interrupt__
Final Analyst Personas:

Name: Dr. Emily Carter
Affiliation: University of Technology
Role: Research Scientist
Description: Dr. Carter focuses on empirical studies of AI frameworks and their impact on computational efficiency and scalability. Her primary concern is how LangGraph can improve the performance of AI systems.
--------------------------------------------------

Name: Alex Johnson
Affiliation: TechInnovate Inc.
Role: CTO
Description: As a CTO, Alex is primarily concerned with the integration of new technologies into existing systems. He is interested in how LangGraph can enhance the functionality and maintainability of TechInnovate's software.
--------------------------------------------------

Name: Samantha Lee
Affiliation: BrightIdeas Startup
Role: CEO
Description: Samantha is an entrepreneur leading a startup that focuses on rapid prototyping and innovation. She is interested in the potential of LangGraph to accelerate produ